## DOC
https://towardsdatascience.com/convolutional-neural-networks-model-from-scratch-for-object-detection-820893610919
https://www.techtarget.com/searchenterpriseai/definition/convolutional-neural-network

In [33]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
class callBackTf(tf.keras.callbacks.Callback):
    def __init__(self, accuracyPersent):
        self.accuracyPersent = accuracyPersent
    def on_epoch_end(self, epoch, logs=[]):
        if (logs.get("accuracy") > self.accuracyPersent):
            print("\n Reach 95% accuracy then stop training")
            self.model.stop_training=True

In [3]:
callBack = callBackTf(accuracyPersent=0.98)

In [13]:
train_dir = 'dataset/horse-or-human/training/'
validation_dir = 'dataset/horse-or-human/validation/'

In [50]:
train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [51]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300,300),
    class_mode='binary'
)
validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(300,300),
    class_mode='binary'
)

Found 256 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [52]:
train_generator.directory

'dataset/horse-or-human/training/'

In [53]:
def normal_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    return model

def CNN_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16,(3,3), activation=tf.nn.relu, input_shape=(300, 300,3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64,(3,3), activation=tf.nn.relu),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    return model

def CNN_model_hourse_human():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16,(3,3), activation=tf.nn.relu, input_shape=(300, 300,3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32,(3,3), activation=tf.nn.relu),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64,(3,3), activation=tf.nn.relu),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64,(3,3), activation=tf.nn.relu),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64,(3,3), activation=tf.nn.relu),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation=tf.nn.relu),
        tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
    ])
    return model

In [54]:
model = CNN_model_hourse_human()

In [55]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [56]:
model.fit(train_generator, epochs=15, validation_data=validation_generator)

Epoch 1/15
8/8 [==============================] - 12s 1s/step - loss: 0.9496 - accuracy: 0.5156 - val_loss: 0.6971 - val_accuracy: 0.5000
Epoch 2/15
8/8 [==============================] - 10s 1s/step - loss: 0.8892 - accuracy: 0.5000 - val_loss: 0.6760 - val_accuracy: 0.4922
Epoch 3/15
8/8 [==============================] - 10s 1s/step - loss: 0.6514 - accuracy: 0.5469 - val_loss: 0.5722 - val_accuracy: 0.7617
Epoch 4/15
8/8 [==============================] - 10s 1s/step - loss: 0.6653 - accuracy: 0.7266 - val_loss: 0.4755 - val_accuracy: 0.8438
Epoch 5/15
8/8 [==============================] - 11s 1s/step - loss: 0.5683 - accuracy: 0.7461 - val_loss: 0.4879 - val_accuracy: 0.8164
Epoch 6/15
8/8 [==============================] - 10s 1s/step - loss: 0.5316 - accuracy: 0.8047 - val_loss: 0.5116 - val_accuracy: 0.8594
Epoch 7/15
8/8 [==============================] - 10s 1s/step - loss: 0.3888 - accuracy: 0.8828 - val_loss: 0.3386 - val_accuracy: 0.8359
Epoch 8/15
8/8 [==================

In [57]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 149, 149, 16)     0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 35, 35, 64)      

In [58]:
path = 'playgoundData/chatchard.jpg'

In [59]:
img = image.load_img(path, target_size=(300,300))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
image_tensor = np.vstack([x])

In [60]:
classes = model.predict(image_tensor)

1/1 [==============================] - 0s 80ms/step


In [61]:
if classes[0] > 0.5:
    print("human")
else:
    print("hrouse")

hrouse
